<a href="https://colab.research.google.com/github/ashwin-yedte/visual-intelligence-travel-finance/blob/main/notebooks/Step_1_Image_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Image Analysis & Feature Extraction

**Version**: 1.0  
**Author**: Ashwin Kumar Y (2023AC05628)  
**Project**: Visual Intelligence for Travel & Finance Optimization  
**Institution**: BITS Pilani  
**Date**: January 2026

---

## Overview

This notebook implements:
- Image preprocessing and validation
- CLIP-based feature extraction (512-dim embeddings)
- Zero-shot scene classification for Indian seashores
- Batch processing for 1-5 images

## Outputs
- Scene classification scores
- Visual descriptors
- Color statistics
- Dominant themes

---

In [3]:
# ============================================================================
# SETUP & INSTALLATION
# ============================================================================

print("Installing required packages...")
print("=" * 70)

# Install dependencies
!pip install -q transformers torch torchvision pillow scikit-learn

print("Installation complete!")
print("=" * 70)

# Import libraries
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
from typing import List, Dict, Tuple
import io
import json
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from google.colab import files
import os
import warnings
warnings.filterwarnings('ignore')

# Check GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\n PyTorch version: {torch.__version__}")
print(f" Device: {device}")
if device == "cuda":
    print(f" GPU: {torch.cuda.get_device_name(0)}")

Installing required packages...
Installation complete!

 PyTorch version: 2.9.0+cpu
 Device: cpu


In [4]:
# ============================================================================
# PROMPT LIBRARY
# ============================================================================

class SeashorePromptLibrary:
    """Collection of prompts for Indian seashore scene classification"""

    PRIMARY_PROMPTS = [
        "a tropical beach with palm trees and golden sand",
        "a pristine white sand beach with clear turquoise water",
        "a rocky coastline with cliffs and crashing waves",
        "a serene beach with calm waters and gentle waves",
        "a beach with traditional fishing boats and nets",
        "a secluded cove with crystal clear water",
        "a sunset beach with orange and pink sky",
        "a beach with water sports activities and equipment",
    ]

    INDIAN_REGIONAL_PROMPTS = [
        "a Goa style beach with colorful shacks and palm trees",
        "a Kerala beach with coconut groves and backwaters",
        "an Andaman island beach with coral reefs and pristine water",
        "a Konkan coast beach with rocky cliffs and coconut trees",
        "a beach with traditional fishing village and local culture",
        "a beach near a temple or coastal religious site",
        "a mangrove lined coastal area with dense vegetation",
        "a Tamil Nadu beach with rocky shores",
    ]

    ACTIVITY_PROMPTS = [
        "a peaceful beach ideal for relaxation and meditation",
        "an adventure beach with water sports and activities",
        "a beach with beach parties and nightlife",
        "a family friendly beach with safe shallow waters",
        "a romantic beach with scenic sunset views",
        "an offbeat secluded beach with minimal crowds",
    ]

    AESTHETIC_PROMPTS = [
        "a beach with dramatic landscape and scenic views",
        "a beach with clear blue sky and bright sunlight",
        "a beach with golden hour lighting and warm tones",
        "a beach with lush green vegetation and natural beauty",
        "a beach with unique rock formations and natural features",
    ]

    @classmethod
    def get_all_prompts(cls) -> List[str]:
        return (cls.PRIMARY_PROMPTS +
                cls.INDIAN_REGIONAL_PROMPTS +
                cls.ACTIVITY_PROMPTS +
                cls.AESTHETIC_PROMPTS)

    @classmethod
    def get_prompt_categories(cls) -> Dict[str, List[str]]:
        return {
            'primary': cls.PRIMARY_PROMPTS,
            'regional': cls.INDIAN_REGIONAL_PROMPTS,
            'activity': cls.ACTIVITY_PROMPTS,
            'aesthetic': cls.AESTHETIC_PROMPTS
        }

# Test
prompts = SeashorePromptLibrary.get_all_prompts()
print(f" Prompt library loaded: {len(prompts)} prompts")
print(f"   - Primary: {len(SeashorePromptLibrary.PRIMARY_PROMPTS)}")
print(f"   - Regional: {len(SeashorePromptLibrary.INDIAN_REGIONAL_PROMPTS)}")
print(f"   - Activity: {len(SeashorePromptLibrary.ACTIVITY_PROMPTS)}")
print(f"   - Aesthetic: {len(SeashorePromptLibrary.AESTHETIC_PROMPTS)}")

 Prompt library loaded: 27 prompts
   - Primary: 8
   - Regional: 8
   - Activity: 6
   - Aesthetic: 5
